# Reducing memory usage of a pandas dataframe

In [6]:
import numpy as np
import pandas as pd

I've used forest cover dataset. It can be downloaded from Kaggle (https://www.kaggle.com/uciml/forest-cover-type-dataset).

In [7]:
df = pd.read_csv('data/covtype.csv')

In [8]:
df.shape

(581012, 55)

In [9]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("--------------------------------")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("-------------------------------")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [10]:
props, NAlist = reduce_mem_usage(df)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 243.8024139404297  MB
--------------------------------
Column:  Elevation
dtype before:  int64
dtype after:  uint16
-------------------------------
--------------------------------
Column:  Aspect
dtype before:  int64
dtype after:  uint16
-------------------------------
--------------------------------
Column:  Slope
dtype before:  int64
dtype after:  uint8
-------------------------------
--------------------------------
Column:  Horizontal_Distance_To_Hydrology
dtype before:  int64
dtype after:  uint16
-------------------------------
--------------------------------
Column:  Vertical_Distance_To_Hydrology
dtype before:  int64
dtype after:  int16
-------------------------------
--------------------------------
Column:  Horizontal_Distance_To_Roadways
dtype before:  int64
dtype after:  uint16
-------------------------------
--------------------------------
Column:  Hillshade_9am
dtype before:  int64
dtype after:  uint8
--------------------------

## Memory usage reduced from 243.8024139404297 MB to 13.863663318591035 MB.